In [1]:
import torch
# GPU 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.cuda.get_device_name(0))

# CUDA 사용 가능 여부 확인
print(f"CUDA available: {torch.cuda.is_available()}")  # True여야 함

# PyTorch에서 사용하는 CUDA 버전 확인
print(f"CUDA version in PyTorch: {torch.version.cuda}")

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cuda
NVIDIA GeForce RTX 4050 Laptop GPU
CUDA available: True
CUDA version in PyTorch: 11.8


In [2]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import albumentations as A
import matplotlib.pyplot as plt
import torch.nn.functional as F

d:\isy\test\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
import torch
import cv2
import numpy as np
import os
import random
from torchvision import models, transforms

# ResNet 및 Inception 모델 수정
class CustomModel(torch.nn.Module):
    def __init__(self, model_name):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.model = models.efficientnet_b2(weights='IMAGENET1K_V1') 
            self.model.classifier[1] = torch.nn.Linear(self.model.classifier[1].in_features, 1)  # 이진 분류
        
    def forward(self, x):
        return self.model(x)

# efficientnet 모델 로드
efficientnet_model = CustomModel(model_name='efficientnet')
efficientnet_model.eval()  # 평가 모드로 설정

# 전처리 파이프라인 정의
preprocess = transforms.Compose([
    transforms.ToPILImage(),  # OpenCV 이미지를 PIL 이미지로 변환
    transforms.Resize((260, 260)),  # 모델 입력 크기에 맞게 크기 조정
    transforms.ToTensor(),  # 텐서로 변환
])

# 이미지 전처리 함수
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환
    image = preprocess(image)  # 전처리 적용
    image = image.unsqueeze(0)  # 배치 차원 추가
    return image

# 예측 함수 정의
def predict_image(model, image_path, device):
    model.to(DEVICE)  # 모델을 디바이스로 이동
    image = preprocess_image(image_path).to(device)  # 전처리 후 디바이스로 이동
    with torch.no_grad():  # 평가 모드에서는 gradient 계산 비활성화
        output = model(image)
        predicted_prob = torch.sigmoid(output).item()  # 확률로 변환

    # 0.5를 기준으로 정상/비정상 예측
    prediction = 'NG (결함 있음)' if predicted_prob > 0.5 else 'OK (정상)'
    return prediction, predicted_prob

# NG 폴더 경로
ng_folder_path = '../PCB_imgs/all/resize/NG/' 
# OK 폴더 경로
ok_folder_path = '../PCB_imgs/all/resize/OK/' 
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 디바이스 설정

# 각 폴더에서 무작위로 10개의 이미지를 가져오기
def load_random_images(folder_path, num_images=10):
    images = os.listdir(folder_path)
    random_images = random.sample(images, min(num_images, len(images)))
    return [os.path.join(folder_path, img) for img in random_images]

# NG 및 OK 이미지 로드
ng_images = load_random_images(ng_folder_path)
ok_images = load_random_images(ok_folder_path)

# NG 이미지 예측 결과와 확률
print(f'NG 예측결과:')
for image_path in ng_images:
    prediction, prob = predict_image(efficientnet_model, image_path, DEVICE) 
    print(f"Efficientnet: {prediction}, 확률: {prob:.4f}")

# OK 이미지 예측 결과와 확률
print(f'OK 예측결과:')
for image_path in ok_images:
    prediction, prob = predict_image(efficientnet_model, image_path, DEVICE) 
    print(f"Efficientnet: {prediction}, 확률: {prob:.4f}")

NG 예측결과:
Efficientnet: OK (정상), 확률: 0.4919
Efficientnet: NG (결함 있음), 확률: 0.5079
Efficientnet: OK (정상), 확률: 0.4975
Efficientnet: OK (정상), 확률: 0.4928
Efficientnet: NG (결함 있음), 확률: 0.5460
Efficientnet: OK (정상), 확률: 0.4424
Efficientnet: NG (결함 있음), 확률: 0.5128
Efficientnet: OK (정상), 확률: 0.4919
Efficientnet: OK (정상), 확률: 0.4974
Efficientnet: NG (결함 있음), 확률: 0.5521
OK 예측결과:
Efficientnet: NG (결함 있음), 확률: 0.5092
Efficientnet: NG (결함 있음), 확률: 0.5270
Efficientnet: NG (결함 있음), 확률: 0.5333
Efficientnet: OK (정상), 확률: 0.4886
Efficientnet: OK (정상), 확률: 0.4970
Efficientnet: NG (결함 있음), 확률: 0.5368
Efficientnet: NG (결함 있음), 확률: 0.5496
Efficientnet: NG (결함 있음), 확률: 0.5065
Efficientnet: NG (결함 있음), 확률: 0.5210
Efficientnet: NG (결함 있음), 확률: 0.5151


In [14]:
from torchvision import datasets
import pandas as pd

IMAGE_SIZE = 260
BATCH_SIZE = 32
EPOCHS = 10

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),  # 이미지를 Tensor로 변환 (0~255 범위를 0~1 범위로 정규화)
])

# 데이터셋 디렉토리 설정
train_dir = '../PCB_imgs/all/resize/train'
val_dir = '../PCB_imgs/all/resize/validation'
test_dir = '../PCB_imgs/all/resize/test'

# ImageFolder로 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# 파일 경로 및 타겟 추출
train_file_paths = [img[0] for img in train_dataset.imgs]
train_targets = train_dataset.targets

val_file_paths = [img[0] for img in val_dataset.imgs]
val_targets = val_dataset.targets

test_file_paths = [img[0] for img in test_dataset.imgs]
test_targets = test_dataset.targets

# DataFrame 생성
train_df = pd.DataFrame({'file_paths': train_file_paths, 'targets': train_targets})
validation_df = pd.DataFrame({'file_paths': val_file_paths, 'targets': val_targets})
test_df = pd.DataFrame({'file_paths': test_file_paths, 'targets': test_targets})

# 확인을 위해 각 데이터셋의 크기 출력
print(f"Train 데이터 수: {len(train_df)}")
print(f"Validation 데이터 수: {len(validation_df)}")
print(f"Test 데이터 수: {len(test_df)}")

Train 데이터 수: 2008
Validation 데이터 수: 502
Test 데이터 수: 628


In [15]:
import torch
from torchvision import datasets, transforms

# resize된 데이터셋 로드
input_dir = '../PCB_imgs/all/resize/'
dataset = datasets.ImageFolder(root=input_dir, transform=transform)

# 데이터 로더 정의
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# 데이터 로더에서 데이터 가져오기
for images, labels in dataloader:
    print(images.shape)  # 배치의 이미지 텐서 크기 확인
    break  # 첫 번째 배치만 확인
    # 배치 크기, 채널 수(RGB), 이미지 크기 

torch.Size([32, 3, 260, 260])


In [16]:
# 커스텀 데이터세트 정의
class CustomDataset(Dataset):
    def __init__(self, file_paths, targets, aug=None, preprocess=None):
        self.file_paths = file_paths
        self.targets = targets
        self.aug = aug
        self.preprocess = preprocess

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        file_path = self.file_paths[index]
        target = self.targets[index]
        
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

        if self.aug is not None:
            image = self.aug(image=image)['image']

        if self.preprocess is not None:
            image = self.preprocess(image)

        image = np.transpose(image, (2, 0, 1))  # (H, W, C) -> (C, H, W)
        image = torch.tensor(image, dtype=torch.float32)
        
        return image, target

In [17]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df['file_paths'].values, train_df['targets'].values)
validation_dataset = CustomDataset(validation_df['file_paths'].values, validation_df['targets'].values)
test_dataset = CustomDataset(test_df['file_paths'].values, test_df['targets'].values)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [18]:
# 평가 함수 정의
def evaluate_model(model, test_loader, criterion):
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images)
            outputs = outputs.view(-1) # torch.Size([batch_size])로 변환
            loss = criterion(outputs, labels)  # 손실 계산
            running_test_loss += loss.item()
            
            predicted = (torch.sigmoid(outputs) > 0.5).float()  # 0.5 기준으로 이진 분류
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
            
    # 검증 손실과 정확도
    test_loss = running_test_loss / len(test_loader)
    test_accuracy = correct_test / total_test
    return test_loss, test_accuracy

efficientnet_b2 모델 사용

In [19]:
import torch
import torch.nn as nn
import torchvision

# 모델 정의
class CustomModel(nn.Module):
    def __init__(self, model_name='efficientnet'):
        super(CustomModel, self).__init__()
        if model_name == 'efficientnet':
            self.base_model = torchvision.models.efficientnet_b2()
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거
        elif model_name == 'resnet50':
            self.base_model = torchvision.models.resnet50()
            self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])  # 마지막 레이어 제거
        elif model_name == 'inception':
            self.base_model = torchvision.models.inception_v3()
            self.base_model.classifier = nn.Identity()  # 마지막 분류기 제거

        self.fc1 = nn.Linear(self._get_features_dim(model_name), 1)

    def _get_features_dim(self, model_name):
        if model_name == 'efficientnet':
            return 1280  # EfficientNet의 출력 차원
        elif model_name == 'resnet50':
            return 2048  # ResNet50의 출력 차원
        elif model_name == 'inception':
            return 2048  # Inception의 출력 차원

    def forward(self, x):
        x = self.base_model(x)
        
        if isinstance(x, tuple): 
            x = x[0]  

        x = x.view(x.size(0), -1)  
        x = self.fc1(x)  

        return x 

In [20]:
# 모델 초기화
model = CustomModel(model_name='efficientnet').to(DEVICE)
# 손실 함수 및 최적화함수 정의
criterion = nn.BCEWithLogitsLoss()  # 이진 교차 엔트로피 손실
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam 옵티마이저

In [21]:
EPOCHS = 20

# 조기 종료 변수를 초기화합니다.
best_val_loss = float('inf')
patience = 5  # 개선이 없을 때 기다릴 에포크 수
patience_counter = 0

# 모델 훈련
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, targets in train_loader:
        images, targets = images.to(DEVICE), targets.to(DEVICE).float()
        
        optimizer.zero_grad()
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.view(-1), targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # 정확도 계산
        predicted = (torch.sigmoid(outputs.view(-1)) > 0.5).float()
        correct_predictions += (predicted == targets).sum().item()
        total_predictions += targets.size(0)

    # 훈련 데이터 정확도
    train_accuracy = correct_predictions / total_predictions

    # 검증
    val_loss, val_accuracy = evaluate_model(model, validation_loader, criterion)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # 손실 개선 시 카운터 리셋
        print("Validation loss improved, saving model...")
    else:
        patience_counter += 1  # 손실이 개선되지 않으면 카운터 증가

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break  # 훈련 종료

OutOfMemoryError: CUDA out of memory. Tried to allocate 40.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 8.73 GiB is allocated by PyTorch, and 288.12 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')